<a href="https://colab.research.google.com/github/danielsuassuna04/TransferLearning-keras/blob/main/TransferLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import tensorflow as tf
from tensorflow import keras

2024-09-19 08:41:56.938386: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-19 08:41:56.938533: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-19 08:41:57.072208: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-19 08:41:57.325217: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-19 08:41:58.987763: W tensorflow/compiler/tf2

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications.resnet import preprocess_input


(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.cifar10.load_data()


batch_size = 32


def preprocess(image, label):
    image = tf.cast(image, tf.float32)
    image = tf.image.resize(image, (224, 224))
    image = preprocess_input(image)
    return image, label
train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
test_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_labels))

train_dataset = (train_dataset
                 .map(lambda x, y: preprocess(x, y))
                 .shuffle(buffer_size=10000)
                 .batch(batch_size)
                 .prefetch(1))
test_dataset = (test_dataset
                .map(lambda x, y: preprocess(x, y))
                .batch(batch_size)
                .prefetch(1))

170498071/170498071 [==============================] - 2s 0us/step


2024-09-19 08:42:05.911697: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-09-19 08:42:06.015849: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-09-19 08:42:06.016012: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [ ]:
base_model = keras.applications.ResNet152(weights="imagenet",include_top=False)
base_model.trainable = False
X = keras.layers.Input(shape=(224,224,3))
x = base_model(X,training=False)
avg = keras.layers.GlobalAveragePooling2D()(base_model.output)
x = keras.layers.Dropout(0.5)(avg)
output = keras.layers.Dense(10, activation="softmax")(x)
model = keras.Model(inputs=base_model.input,outputs=output)

In [ ]:
optimizer = keras.optimizers.Adam(learning_rate=0.01,weight_decay=0.01)
model.compile(loss="sparse_categorical_crossentropy",optimizer=optimizer,metrics=["accuracy"])
early = keras.callbacks.EarlyStopping(patience=5,restore_best_weights=True)
checkpoint = keras.callbacks.ModelCheckpoint(
    filepath='best_model.weights.h5',
    monitor='val_loss',
    save_best_only=True,
    mode='min',
    save_weights_only=True,
    verbose=1
)
history = model.fit(train_dataset,epochs=5,validation_data=test_dataset,callbacks=[early,checkpoint])

Epoch 1/5
1563/1563 [==============================] - ETA: 0s - loss: 1.7845 - accuracy: 0.8339
Epoch 1: val_loss improved from inf to 1.52440, saving model to best_model.weights.h5
1563/1563 [==============================] - 231s 141ms/step - loss: 1.7845 - accuracy: 0.8339 - val_loss: 1.5244 - val_accuracy: 0.8788
Epoch 2/5
1563/1563 [==============================] - ETA: 0s - loss: 2.0809 - accuracy: 0.8606
Epoch 2: val_loss improved from 1.52440 to 1.40020, saving model to best_model.weights.h5
1563/1563 [==============================] - 217s 136ms/step - loss: 2.0809 - accuracy: 0.8606 - val_loss: 1.4002 - val_accuracy: 0.8985
Epoch 3/5
1563/1563 [==============================] - ETA: 0s - loss: 2.1466 - accuracy: 0.8657
Epoch 3: val_loss did not improve from 1.40020
1563/1563 [==============================] - 211s 132ms/step - loss: 2.1466 - accuracy: 0.8657 - val_loss: 1.4448 - val_accuracy: 0.9006
Epoch 4/5
1563/1563 [==============================] - ETA: 0s - loss: 2.22

In [ ]:
model.load_weights("best_model.weights.h5")

In [ ]:
model.evaluate(test_dataset)

313/313 [==============================] - 34s 108ms/step - loss: 1.3802 - accuracy: 0.9106


[1.3801923990249634, 0.9106000065803528]